In [ ]:
import pandas as pd

In [ ]:
dataset = pd.read_csv('dataset_output.csv')

In [ ]:
dataset.shape

(91782, 1469)

In [ ]:
dataset.tail(5)

,customer_ID,P_2_Ave_6,P_2_Min_6,P_2_Max_6,P_2_Ave_12,P_2_Min_12,P_2_Max_12,D_39_Ave_6,D_39_Min_6,D_39_Max_6,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,months_of_data
91777,fb1c800a74828eba6431ebaf63fdee04f068d1508c79ab...,0.611574,0.567507,0.710410,0.304045,0.203019,0.367251,0.003979,0.002519,0.005824,...,NaN,NaN,0.003069,0.000916,0.009173,NaN,0.003071,0.007909,0.008699,13.0
91778,66d3fc4fe4593996fdc5791295a2217fec1f54270f838c...,0.310654,0.261535,0.449123,0.910926,0.871052,0.928140,0.004730,0.001574,0.006894,...,NaN,NaN,1.001755,0.005416,0.858056,0.012321,1.007006,0.002519,0.095377,13.0
91779,9f45276d7e50097fa272250545ddf98272c5031f5ef131...,0.901136,0.883907,0.914022,0.120808,-0.041712,0.225689,0.171976,0.002466,0.566619,...,NaN,NaN,0.009421,0.001896,0.005171,NaN,0.003437,0.001484,0.004568,13.0
91780,062ea289d96896b2d0b23f77b7c22d1764c40b7a4ae0c4...,0.497986,0.332845,0.622056,0.920031,0.907405,0.930120,0.187727,0.000028,0.391132,...,NaN,NaN,0.005817,0.005515,0.009270,NaN,0.005973,0.006140,0.008742,3.0
91781,314708335a3dd5faa2656f9e88bce41ed4f95263d8d30a...,0.614203,0.581069,0.624880,0.981046,0.957823,1.001323,0.162325,0.000474,0.415258,...,NaN,NaN,0.006574,0.003428,0.002802,NaN,0.008710,0.008416,0.003380,13.0


In [ ]:
dataset.columns

Index(['customer_ID', 'P_2_Ave_6', 'P_2_Min_6', 'P_2_Max_6', 'P_2_Ave_12',
       'P_2_Min_12', 'P_2_Max_12', 'D_39_Ave_6', 'D_39_Min_6', 'D_39_Max_6',
       ...
       'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143', 'D_144',
       'D_145', 'months_of_data'],
      dtype='object', length=1469)

In [ ]:
dataset['target']

,target
0,1
1,0
2,0
3,1
4,0
...,...
91777,0
91778,1
91779,0
91780,0


In [ ]:
dataset.isnull().sum()

,0
customer_ID,0
P_2_Ave_6,555
P_2_Min_6,555
P_2_Max_6,555
P_2_Ave_12,502
...,...
D_142,77547
D_143,9942
D_144,6162
D_145,9942


In [ ]:
# Chia dataset thành các cột số và cột chuỗi
numeric_cols = dataset.select_dtypes(include=['number']).columns
categorical_cols = dataset.select_dtypes(include=['object']).columns


In [ ]:
from sklearn.impute import SimpleImputer

# Áp dụng SimpleImputer cho các cột số
numeric_imputer = SimpleImputer(strategy='mean')
dataset[numeric_cols] = numeric_imputer.fit_transform(dataset[numeric_cols])


In [ ]:
# Áp dụng SimpleImputer cho các cột chuỗi, thay thế NaN bằng giá trị phổ biến nhất
categorical_imputer = SimpleImputer(strategy='most_frequent')
dataset[categorical_cols] = categorical_imputer.fit_transform(dataset[categorical_cols])


In [ ]:
!pip install scikit-learn


In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
# Tách dữ liệu thành X (features) và y (target)
X = dataset.drop(['customer_ID', 'target'], axis=1)  # Loại bỏ cột không cần thiết
y = dataset['target']  # Cột đích (target)


In [ ]:
from sklearn.model_selection import train_test_split

# Chia dữ liệu thành tập huấn luyện và kiểm tra (70% huấn luyện, 30% kiểm tra)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
# Chuyển cột S_2 từ kiểu chuỗi (ngày tháng) thành kiểu số (số ngày kể từ ngày 1 tháng 1 năm 1970)
X_train['S_2'] = pd.to_datetime(X_train['S_2'])
X_test['S_2'] = pd.to_datetime(X_test['S_2'])

# Tính số ngày kể từ ngày 1 tháng 1 năm 1970
X_train['S_2'] = (X_train['S_2'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1D')
X_test['S_2'] = (X_test['S_2'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1D')


In [ ]:
from imblearn.over_sampling import SMOTE

# Áp dụng SMOTE cho tập huấn luyện để cân bằng dữ liệu
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


In [ ]:
import pandas as pd

# Chuyển đổi X_train_resampled và y_train_resampled thành DataFrame
X_train_resampled_df = pd.DataFrame(X_train_resampled, columns=X_train.columns)  # nếu X_train là DataFrame
y_train_resampled_df = pd.DataFrame(y_train_resampled, columns=['target'])

# Gộp X_train_resampled với y_train_resampled thành một DataFrame
resampled_data = pd.concat([X_train_resampled_df, y_train_resampled_df], axis=1)

# Lưu vào file CSV
resampled_data.to_csv('balanced_data.csv', index=False)

print("Đã lưu dữ liệu đã cân bằng vào file 'balanced_data.csv'.")


Đã lưu dữ liệu đã cân bằng vào file 'balanced_data.csv'.
